# YOLOV3 object detection

In this example, we'll demonstrate YOLOV3 object detection using OpenCV and the pretrained DarkNet model, provided by the author of YOLO. The model is trained using the COCO dataset.

In [14]:
import os
import cv2
import numpy as np
import requests

ImportError: cannot import name 'get_host' from 'urllib3.util.url' (C:\Users\delaplai\git\learning\Advanced-Deep-Learning-with-Python\venv\lib\site-packages\urllib3\util\url.py)

Add some bilderplate code that downloads and stores several configuration and data files. W will start with the YOLOv3 network configuration ylo_config and weights, and we will us ethem to initalize the net network

In [15]:
# Download YOLO net config file
yolo_config = "yolov3.cfg"
id not os.path.isfile(yolo_config):
    url = "https://raw"

SyntaxError: invalid syntax (85865275.py, line 3)